# Linear least squares, Lasso, and ridge regression
## 理解回归
回归主要关注一个唯一的因变量（需要预测的值）和一个或多个数值型的自变量（预测变量）之间的关系。我们首先假设因变量和自变量之间的关系遵循一条直线，即线性关系。我们可以用这样的函数来表示
$ \begin{equation}
    f(v) := \lambda\, R(w) +
    \frac1n \sum_{i=1}^n L(v;x_i,y_i)
    \label{eq:regPrimal}
    \ .
\end{equation} $。
其中L(w;x,y)可以表示成 $y=w^Tx$,$\lambda\, R(w)$是正则化表达式。  
所以我们可以这样定义损失函数：  
$$L(w;x,y) :=  \frac{1}{2} (w^T x - y)^2$$  
为了找到一组参数使得损失函数L(w;x,y)取的最小值，我们可以使用最小二乘法。  
> 不同的回归方法使用不同的正则化普通最小二乘或线性最小二乘不使用正规化，ridge regression 使用的是L2正则化; Lasso 使用的是 L1 正则化。

## 例子
这个例子使用LinearRegressionWithSGD 去构建一个简单的线性预测标签值。

In [2]:
import org.apache.spark.mllib.regression.LabeledPoint
import org.apache.spark.mllib.regression.LinearRegressionModel
import org.apache.spark.mllib.regression.LinearRegressionWithSGD
import org.apache.spark.mllib.linalg.Vectors

val PATH = "file:///Users/lzz/work/SparkML/"
// 加载并解析数据
val data = sc.textFile( PATH+"data/mllib/ridge-data/lpsa.data")
val parsedData = data.map { line =>
  val parts = line.split(',')
  LabeledPoint(parts(0).toDouble, Vectors.dense(parts(1).split(' ').map(_.toDouble)))
}.cache()

// 建立模型
val numIterations = 100
val model = LinearRegressionWithSGD.train(parsedData, numIterations)

// 在样本中训练模型并且计算训练误差
val valuesAndPreds = parsedData.map { point =>
  val prediction = model.predict(point.features)
  (point.label, prediction)
}
val MSE = valuesAndPreds.map{case(v, p) => math.pow((v - p), 2)}.mean()
println("training Mean Squared Error = " + MSE)

// 保存和加载模型
model.save(sc, "myModelPath")
val sameModel = LinearRegressionModel.load(sc, "myModelPath")

training Mean Squared Error = 6.207597210613578


RidgeRegressionWithSGD and LassoWithSGD can be used in a similar fashion as LinearRegressionWithSGD.

In order to run the above application, follow the instructions provided in the Self-Contained Applications section of the Spark quick-start guide. Be sure to also include spark-mllib to your build file as a dependency.